# Calling functions and Procedures in Snowpark

We will perform aggregate and set-based operations on dataframes

1. Function
2. Table Functions
3. UDFs

For more information follow ths below links:
1. [Calling Functions and Stored Procedures in Snowpark Python](https://docs.snowflake.com/en/developer-guide/snowpark/python/calling-functions)

2. [CREATE FUNCTION](https://docs.snowflake.com/en/sql-reference/sql/create-function#python-handler)

3. [CREATE PROCEDURE](https://docs.snowflake.com/en/sql-reference/sql/create-procedure#python-handler)

In [ ]:
from snowflake.snowpark.functions import (col, lit, upper,
    call_udf, udtf, function, call_function, call_table_function)
from snowflake.snowpark.types import StructType, StructField, StringType
from snowflake.snowpark.context import get_active_session

In [ ]:
session = get_active_session()
df = session.create_dataframe(
    [[1, 'John Doe'], [2, 'Mary Poppins']],
    schema=["id", "name"])
df

In [ ]:
#Call Snowpark Wrapper to built in SQL Function

df.select("id", upper(col("name")).alias("upper_name"))


In [ ]:
#Call Explicit wrapper to built in SQL function
df.select("id", function("upper")(col("name")).alias("upper_name"))

In [ ]:
#Call regiseterd stored proc
session.call("snowpark_db.sourced.procPython",1)

In [ ]:
df.select(call_udf("snowpark_db.sourced.UdfPython",col("id")).alias("udf"),"name")

In [ ]:
# define new UDTF handler
class MyClass:
    def process(self, s):
        yield (s, )
        yield (s, )

# create new anonymous UDTF + call it
udtfPython = udtf(MyClass,
    output_schema=StructType([StructField("s", StringType())]),
    input_types=[StringType()])
session.table_function(udtfPython(lit("abc")))

In [ ]:
# call built-in SQL table function on constant value
session.table_function("split_to_table", lit("John Doe"), lit(" "))

In [ ]:
# call built-in SQL table function on constant value
session.table_function(call_table_function("split_to_table", lit("John Doe"), lit(" ")))

In [ ]:
# call built-in SQL table function on table values
df.join_table_function("split_to_table", col("name"), lit(" "))